In [ ]:
import pandas as pd

### Read in the original DataSet for comparsion:

In [ ]:
original = pd.read_csv("../data/lrpd.csv")
print(f"Successfully read in Original Little Rock Crime Data. DataFrame Size: {original.shape[0]} rows x {original.shape[1]} cols")

### Request relevant columns from dataset:

In [ ]:
requested_columns = ["INCIDENT_DATE", "INCIDENT_NUMBER", "LOCATION_DISTRICT", "OFFENSE_DESCRIPTION", "WEAPON_TYPE", "ZIP", "LATITUDE", "LONGITUDE"]

lrpd = pd.read_csv("../data/lrpd.csv", usecols=requested_columns, index_col="INCIDENT_NUMBER")

print(f"Successfully read in Little Rock Crime Data. DataFrame Size: {lrpd.shape[0]} rows x {lrpd.shape[1]} cols")

### Drop duplicate rows based on index values:

In [ ]:
prev_shape = lrpd.shape
lrpd = lrpd.drop_duplicates(keep='first')
new_shape = lrpd.shape

print(f'Dropped {(prev_shape[0] - new_shape[0])} duplicates!')
print(f'Little Rock Crime Data DataFrame Size: {lrpd.shape[0]} rows x {lrpd.shape[1]} cols')

### Fill missing values with Number 0:

In [ ]:
lrpd = lrpd.fillna(0)

### Parse Number 0 and String 1 to weapon type:

In [ ]:
lrpd["WEAPON_TYPE"] = lrpd["WEAPON_TYPE"].replace(0, "NO WEAPON")
lrpd["WEAPON_TYPE"] = lrpd["WEAPON_TYPE"].replace("1", "UNKNOWN")

### Convert columns to correct types:

In [ ]:
lrpd["INCIDENT_DATE"] = pd.to_datetime(lrpd["INCIDENT_DATE"])
lrpd["LOCATION_DISTRICT"] = pd.to_numeric(lrpd["LOCATION_DISTRICT"])
lrpd["OFFENSE_DESCRIPTION"] = lrpd["OFFENSE_DESCRIPTION"] # todo: ensure this is parsed as a string
lrpd["WEAPON_TYPE"] = lrpd["WEAPON_TYPE"] # todo: ensure this is parsed as a string
lrpd["ZIP"] = pd.to_numeric(lrpd["ZIP"])
lrpd["LATITUDE"] = pd.to_numeric(lrpd["LATITUDE"])
lrpd["LONGITUDE"] = pd.to_numeric(lrpd["LONGITUDE"])

lrpd.head().T

### Appending new columns to DataFrame:

In [ ]:
lrpd["WEEK_OF_MONTH"] = lrpd["INCIDENT_DATE"].dt.day // 7
lrpd["YEAR"] = lrpd["INCIDENT_DATE"].dt.year
lrpd["DAY"] = lrpd["INCIDENT_DATE"].dt.day
lrpd["DAY_OF_YEAR"] = lrpd["INCIDENT_DATE"].dt.dayofyear
lrpd["MONTH"] = lrpd["INCIDENT_DATE"].dt.month

### Visualizing current modifications:

In [ ]:
lrpd.head().T

In [ ]:
lrpd.tail().T

### Determining crime risk and violence level:

In [ ]:
all_crimes = lrpd["OFFENSE_DESCRIPTION"].unique()
all_weapons = lrpd["WEAPON_TYPE"].unique()

Pre-defining which crimes are considered violent and nonviolent:

In [ ]:
violent_crimes = ['RAPE', 'AGGRAVATED ASSAULT', 'ALL OTHER LARCENY', 'ROBBERY', 'BURGLARY/B&E',
                  'MURDER & NONNEGLIGENT MANSLAUGHTER']
nonviolent_crimes = ['THEFT FROM MOTOR VEHICLE', 'MOTOR VEHICLE THEFT', 'THEFT OF MOTOR VEHICLE PARTS',
                     'SHOPLIFTING', 'THEFT FROM BUILDING', 'POCKET-PICKING', 'THEFT FROM COIN-OPERATED MACHINE',
                     'PURSE-SNATCHING']

In [ ]:
def determine_crime_type(crime):
    if crime in violent_crimes:
        return 'Violent Crime'
    elif crime in nonviolent_crimes:
        return 'Non-Violent Crime'
    else:
        return 'Crime Type Unknown'

In [ ]:
def determine_risk_type(crime):
    # todo: turn this into a function that performs risk assessment
    pass

In [ ]:
lrpd["CRIME_TYPE"] = lrpd["OFFENSE_DESCRIPTION"].apply(determine_crime_type)

In [ ]:
lrpd.loc[(lrpd['CRIME_TYPE'] == 'Violent Crime') | (
        lrpd['WEAPON_TYPE'] != 'NO WEAPON'), "RISK_TYPE"] = "High Risk"

lrpd.loc[(lrpd['CRIME_TYPE'] == 'Violent Crime') & (
        lrpd['CRIME_TYPE'] != 'NO WEAPON'), "RISK_TYPE"] = "High Risk"

lrpd.loc[(lrpd['CRIME_TYPE'] == 'Non-Violent Crime') & (
        lrpd['WEAPON_TYPE'] != 'NO WEAPON'), "RISK_TYPE"] = "High Risk"

lrpd.loc[(lrpd['CRIME_TYPE'] == 'Non-Violent Crime') & (
        lrpd['WEAPON_TYPE'] == 'NO WEAPON'), "RISK_TYPE"] = "Low Risk"

In [ ]:
lrpd['RISK_TYPE_BC'] = lrpd["RISK_TYPE"].map({'High Risk': 1, 'Low Risk': 0})

### Visualizing current modifications:

In [ ]:
lrpd.head().T

In [ ]:
lrpd.tail().T

### Saving the modified DataSet to a new CSV:

In [ ]:
lrpd.to_csv("../data/lrpd-clean.csv")

In [ ]:
lrpd.shape